In [ ]:
## Hit Run > Run all cells!

import turtle as t
from time import sleep
import time
import random as rand
from math import *
from copy import copy, deepcopy

In [ ]:
direction_set = {
    'f': (lambda x,scale: t.forward(x*1)),
    'b': (lambda x,scale: t.backward(x*1)),
    'r': (lambda x,scale: t.right(x)),
    'l': (lambda x,scale: t.left(x)),
}

In [ ]:
# Returns the distance of a fractal
def total_distance(input_string):
    distance = 0
    elements = input_string.split(' ')
    for element in elements:
        if element[0] in "fb":
            distance += float(element[1:])
    return distance

In [ ]:
# Returns the type of a single step of a fractal
def direction_type(direction):
    # Example direction: "f200"
    movements = ['f','b']
    rotations = ['r','l']
    goto = ['G']
    reset = ['R']
    
    d = direction[0]
    if d in movements:
        return "movement"
    elif d in rotations:
        return "rotation"
    elif d in goto:
        return 'goto'
    elif d in reset:
        return 'reset'
    
    return None

In [ ]:
# Gets the directions as a list of (str, float)
def read_directions(input_string):
    return [(subs[0], float(subs[1:])) for subs in input_string.split(' ')]

# Reverses read_directions
def write_directions(input_list):
    return ' '.join([str(a) + str(b) for a,b in input_list])

In [ ]:
# Scales a fractal, but ONLY THE MOVEMENT INSTRUCTIONS, not the rotational ones
def scale_directions(input_string, scale):
    directions = read_directions(input_string)
    scaled_directions = []
    for a,b in directions:
        if a in ['f','b']:
            b *= scale
        scaled_directions.append((a,b))
    return write_directions(scaled_directions)

In [ ]:
# Creates the fractal. "eval_dir" => "evaluate_directions"
def eval_dir(start_time, input_string,
             depth, seed, coloring, colorset, 
             cycle_duration):
    
    kwargs_local = locals()
    rand.seed(seed)
    rand.shuffle(colorset)
    
    # Code for how to color the fractal
    if coloring == 'depth':
        t.pencolor(colorset[depth % len(colorset)])
    elif coloring == 'time':
        time_passed = start_time - time.time()
        index = round((time_passed/cycle_duration)) % len(colors)
        t.pencolor(colorset[index])
    elif coloring == 'none':
        pass
       
    # Depth check
    if depth == 0:
        return 0
    
    # Processes the directions
    directions = [(subs[0], float(subs[1:])) for subs in input_string.split(' ')]
    length = None
    
    # Old code for recursive step
    """
    for func, arg in directions:
        direction_set[func](arg,scale)
        
        kwargs_local["scale"] = scale*delta
        kwargs_local["depth"] = depth - 1
        eval_dir(**kwargs_local)
    """
    
    # If final iteration, do all directions and then stop
    if depth == 1:
        for func, arg in directions:
            direction_set[func](arg,1)
        return 0
    
    # Recursive steps
    # RULES:
    # - No repeat when fractal is rotated
    # - Repeat only on movement
    for func, arg in directions:
        current_step = str(func) + str(arg)
        if direction_type(current_step) == "movement":
            current_step_distance = arg
            
            # Go one step deeper
            # scale = current_step_distance/total_sequence_length
            scale_ = current_step_distance/total_distance(input_string)
            kwargs_local["input_string"] = scale_directions(input_string, scale_)
            kwargs_local["depth"] = depth - 1
            eval_dir(**kwargs_local)
        else:
            direction_set[func](arg,1)

            
def turtle_init(x,y):
    t.reset()
    #-450, 0
    t.goto(x,y)

    t.delay(0)
    t.bgcolor("black")
    t.color("cyan")
    
    
def run_eval_dir(kwargs):
    kwargs2 = copy(kwargs)
    kwargs2["input_string"] = scale_directions(kwargs["input_string"], kwargs["scale"])
    kwargs2.pop("scale",None)
    kwargs2.update({"start_time" : time.time()})
    eval_dir(**kwargs2)
    

In [ ]:
def generate_fractal(fractal):
    """Generates a fractal, given an input as a dictionary of kwargs.
    
    ~  input_string: The directions for the fractal
    ~  depth: How deep to go/how many copies to make
    ~  seed: Seed for the random number generation (affects coloration)
    ~  coloring: The type of coloring - ["time", "depth", "none"]
    ~  cycle_duration: How long each timecycle takes (if coloring = "time")
    ~  colorset: The ordered set of colors to use for coloring
    ~  scale: How much to scale down the fractal at the next depth 
    
    # Side note: The time to generate the fractal increases
                      exponentially with depth.
    
    example_fractal = {
        "input_string" : "f200 l60 f200 r60 r60 f200 l60 f200",
        "depth" : 4,
        "seed" : 5,
        "coloring" : "none",
        "cycle_duration" : 0.2,
        "colorset" : colors,
        "scale" : 3,
    }\n"""
    run_eval_dir(fractal)
    

In [ ]:
# Some pre-created fractals
# If a fractal takes too long to generate, do Kernel > Interrupt kernel to halt

colors = ["red","yellow","green","blue","purple","orange"]
colors2 = ["red","yellow","green","blue","green","blue"]

example_fractal = {
    "input_string" : "f200 l60 f200 r60 r60 f200 l60 f200",
    "depth" : 4, 
    "seed" : 5,
    "coloring" : "none",
    "cycle_duration" : 0.2,
    "colorset" : colors,
    "scale" : 3,
}

koch_snowflake = {
    "input_string" : "f200 l60 f200 r60 r60 f200 l60 f200",
    "depth" : 5,
    "seed" : 5,
    "coloring" : "none",
    "cycle_duration" : 0.2,
    "colorset" : colors,
    "scale" : 4,
}

hex_ = {
    "input_string" : "f200 r60 f200 l60 f200 l60 f200 r60 f200",
    "depth" : 1,
    "seed" : 5,
    "coloring" : "none",
    "cycle_duration" : 0.1,
    "colorset" : colors,
    "scale" : 1,
}

bat = {
    "input_string" : ' r60 '.join("f200 l60 f200 r60 r60 f200 l60 f200 l270 f400 l180 f400 l180 f300 r180 f100 r180 l45".split(' ')),
    "depth" : 3,
    "seed" : 5,
    "coloring" : "time",
    "cycle_duration" : 0.2,
    "colorset" : colors,
    "scale" : 10,
}

In [ ]:
# Run me!
print(generate_fractal.__doc__)
turtle_init(*(-450,0))
generate_fractal(koch_snowflake)   # Call this function to generate a fractal